In [17]:
import pandas as pd
from fun import *

In [18]:
df = pd.read_csv("data/translations.csv")
df.head()

,lp,src,mt,ref,score,raw,annotators,domain,year,l1,l2
0,en-zh,Police said in a statement at the time that th...,警方在当时的一份声明中表示，他们决定以“沉重的心情”结束对 Jacsun 的搜索，但指出他们...,警方在当时的一份声明中表示，他们在决定结束对杰森的搜寻时，是怀着“沉重的心情”的，并指出他们...,0.238989,80.500000,4,news,2020,en,zh
1,en-zh,"Trump will likely veto the resolution, the six...",特 朗 普 可 能 会 否 决 该 决 议 ， 这 是 他 第 六 次 以 总 统 身 份 ...,特朗普很可能行使总统否决权，这将是他成为总统以来第6次否决国会立法。他早在三月已动用否决权，...,0.487842,81.166667,6,news,2020,en,zh
2,en-zh,The man arrived at work about 9.30am after fai...,该名男子未能回复同事的几条短信和电话，于上午9.30左右上班。,该男子大约在上午 9:30 到达工作单位，在此之前，他未能回复同事发来的几条信息和打来的电话。,0.469850,80.500000,4,news,2020,en,zh
3,en-zh,Are hopes for a nuclear-free world realistic?,对一个无核世界的希望是现实的吗 ？,实现无核化世界的希望是否现实？,0.626218,81.000000,4,news,2020,en,zh
4,en-zh,The California attorney general's office in Ma...,"3月,加利福尼亚总检察长办公室在经过近一年的调查后拒绝提出州刑事指控,当时,总检察长Xavi...",3 月，加州总检察长办公室在完成了近一年的案件调查后，拒绝对两名警察提起加州刑事起诉。总检察...,-0.207602,59.000000,4,news,2020,en,zh


In [19]:
with open("data/prompts/PROMPT2.txt", 'r') as f:
    prompt_template = f.read()

print(prompt_template)

Please evaluate the quality of the translation from [ORIGINAL LANGUAGE] to [TRANSLATED LANGUAGE]:

Original text: "[ORIGINAL TEXT]"
Translated text: "[TRANSLATED TEXT]"

Provide a score from 0 to 100 to indicate the quality of the translation, there is no need of adding an explanition, just return the score.


In [20]:
def generate_mapper(row):
  return {
        "[ORIGINAL LANGUAGE]": row["l1"],
        "[TRANSLATED LANGUAGE]": row["l2"],
        "[ORIGINAL TEXT]": row["src"],
        "[TRANSLATED TEXT]": row["mt"],
    }

In [21]:
def get_prompt(prompt_template: str, mapper: dict):
    prompt = prompt_template
    for tag, value in mapper.items():
        prompt = prompt.replace(tag, value)
    
    return prompt

In [22]:
df["prompt"] = df.apply(lambda row: get_prompt(prompt_template, generate_mapper(row)), axis=1)

In [23]:
print(df["prompt"][0])

Please evaluate the quality of the translation from en to zh:

Original text: "Police said in a statement at the time that they made the decision to end the search for Jacsun with a "heavy heart," but noted they couldn't pinpoint a location in the landfill "to a point that would make continuing the search reasonable.""
Translated text: "警方在当时的一份声明中表示，他们决定以“沉重的心情”结束对 Jacsun 的搜索，但指出他们无法确定垃圾填埋场的位置，“以至于继续搜索是合理的”。"

Provide a score from 0 to 100 to indicate the quality of the translation, there is no need of adding an explanition, just return the score.


In [26]:
from mistralai.client import MistralClient
from dotenv import load_dotenv
import os

load_dotenv()
MISTRAL_API = os.getenv("MISTRAL_API")
client = MistralClient(api_key=MISTRAL_API)

In [28]:
send_message_mistral(client, df["prompt"][0])

'I would give the translation a score of 100. The meaning of the original text has been accurately conveyed in the translated text. The use of similar phrases and structures in both the original and translated text further strengthens the quality of the translation.'